# Validación cruzada

Hasta ahora hemos aprendido la _validacion de entrenamiento/prueba_. Ahora nos concentraremos en técnicas más robustas. <br>

### Validacion de retención (_Holdout validation_).

La cual consiste en:
- Dividir el conjunto de datos completo en 2 particiones:
    - Un conjunto de entrenamiento.
    - Un conjunto de pruebas
    - Entrenar el modelo con el conjunto de entrenamiento.
- Utilizar el modelo entrenado para predecir etiquetas en el conjunto de pruebas.
- Calcular una métrica de error para comprender la eficacia del modelo.
- Cambiar los conjuntos de entrenamiento y de prueba y repetirlo.
- Promediar errores.

Nota: Con la validación _**holdout validation**_, se suele utilizar una division de 50/50 en lugar de 75/25 de la validación de entrenamiento/prueba. 

In [1]:
import numpy as np
import pandas as pd

# Leyendo datos
dc_listings = pd.read_csv('./dc_airbnb.csv')

# Limpiando y preparando columna de interes.
stripped_commas = dc_listings['price'].str.replace(',','')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

# Ordenando aleatoriamente los datos
print(dc_listings.index)
shuffled_index = np.random.permutation(dc_listings.index)
dc_listings = dc_listings.reindex(shuffled_index)
dc_listings.head()

RangeIndex(start=0, stop=3723, step=1)


,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state
1259,NaN,NaN,1,3,Entire home/apt,0.0,1.0,1.0,101.0,NaN,NaN,2,1125,0,38.910406,-77.039755,Washington,20036,DC
3169,NaN,0%,1,3,Entire home/apt,2.0,2.5,1.0,139.0,$75.00,NaN,2,1125,5,38.926106,-77.033078,Washington,20009,DC
1348,100%,95%,3,4,Entire home/apt,1.0,1.5,1.0,98.0,$75.00,$500.00,4,1125,2,38.916129,-77.017017,Washington,20001,DC
1069,100%,100%,3,2,Entire home/apt,1.0,1.0,1.0,145.0,$65.00,$250.00,3,1125,60,38.914700,-77.040201,Washington,20009,DC
1158,80%,100%,2,3,Entire home/apt,0.0,1.0,1.0,300.0,$50.00,$300.00,2,1125,14,38.909466,-77.045330,Washington,20036,DC


In [2]:
#################################################
# Dividiendo los datos para la validacion cruzada
#################################################

split_one = dc_listings.iloc[0:1862].copy()
split_two = dc_listings.iloc[1862:].copy()

In [3]:
#################################################
# Entrenando modelos 
#################################################

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Asignando datos de entrenamiento y prueba
train_one = split_one
test_one = split_two
train_two = split_two
test_two = split_one

# Creacion de modelo KNN, ajuste y prediccion para train y test one.
knn = KNeighborsRegressor(n_neighbors=5, algorithm='auto')
knn.fit(train_one[ ['accommodates'] ], train_one['price'])
test_one['predicted_price'] = knn.predict( test_one[ ['accommodates'] ] )
iteration_one_rmse = mean_squared_error(test_one['price'], test_one['predicted_price']) ** (1/2)

# Creacion de modelo KNN, ajuste y prediccion para train y test two
knn = KNeighborsRegressor(n_neighbors=5, algorithm='auto')
knn.fit(train_two[ ['accommodates'] ], train_two['price'])
test_two['predicted_price'] = knn.predict( test_two[[ 'accommodates' ]] )
iteration_two_rmse = mean_squared_error(test_two['price'], test_two['predicted_price']) ** (1/2)

In [4]:
#####################################
# Calculando el promedio de los RMSE
#####################################

avg_rmse = np.mean([iteration_one_rmse, iteration_two_rmse])

print(f'RMSE_one = {iteration_one_rmse}\nRMSE_two = {iteration_two_rmse}\nAVG = {avg_rmse}')

RMSE_one = 130.3826267589315
RMSE_two = 118.75447525062606
AVG = 124.56855100477878


### Validacion cruzada K-Fold

La _validacion holdout_ es en realidad un ejemplo específico de una clase mas amplia de técnicas de validacion llamada _**validación cruzada k-fold (K-fold cross-validation)**_. Esta validacion aprovecah una mayor proporcion de los datos durante el entrenamiento mientras sigue rotando a través de diferentes subconjuntos de los datos. <br>

La cual consiste en:
- Dividir el conjunto de datos en **K** particiones de igual longitud.
    - Seleccionar **K-1** particiones como conjunto de entrenamiento.
    - Seleccionar la partción restante como conjunto de prueba.
- Entrenamiento del modelo con el conjunto de entrenamiento.
- Utilizar el modelo entrenaod para predecir etiquetas en el conjunto de prueba.
- Calcular la metrica de error del conjunto de prueba.
- Repetir todos los pasos anteriores **K-1** veces, hasta que cada partición se haya utilizado como conjunto de prueba para una iteración.
- Calcular la media de los **K** valores de error.  

In [7]:
################################
# Generando pligues manualmente
################################

dc_listings.loc[ dc_listings.index[0:745], 'fold' ] = 1
dc_listings.loc[ dc_listings.index[745:1490], 'fold' ] = 2
dc_listings.loc[ dc_listings.index[1490:2234], 'fold' ] = 3
dc_listings.loc[ dc_listings.index[2234:2978], 'fold' ] = 4
dc_listings.loc[ dc_listings.index[2978:3723], 'fold' ] = 5

# Verificando pliegues
print( dc_listings['fold'].value_counts() )

# Verificando si existen valores perdidos
null_values = dc_listings['fold'].isnull().sum()
print(f'\nNumero de valores perdidos: {null_values}')

fold
1.0    745
2.0    745
5.0    745
3.0    744
4.0    744
Name: count, dtype: int64

Numero de valores perdidos: 0


### IMPORTANTE

Un **RMSE** más bajo no siempre significa que un modelo es mas preciso. Un modelo tiene dos fuentes de error, _**el sesgo y la varianza**_.<br>

El _**sesgo**_ describe el error que resulta de las malas susposiciones sobre el algoritmo de aprendizaje. Por ejemplo, suponer que solo una característica, como el peso de un coche, está relacionada con la eficiencia del combustible. La tasa de error será alta, ya que la eficiencia de combustible se ve afectada por muchos otros factores.<br>

La _**varianza**_ describe el error que se produce debido a la variabilidad de los valores predichos de un modelo. Por ejemplo, si nos dieran un conjunto de datos con 1000 caracteristicas de cada coche y utilizáramos cada una de ellas para entrenar el modelo, trendriamos un sesgo bajo pero una varianza alta.<br>

En un _mundo ideal_, queremos un sesgo bajo y una variaza baja, pero en realidad,  siempre hay una compensación.<br>

La **desviación estándar de los valores de RMSE** puede ser un indicador de la _varianza_ del modelo, mientras que **el RMSE medio** es un indicador del _sesgo_ de un modelo.<br>

El _**sesgo**_ y la _**varianza**_ son las dos fuentes de error observables en un modelo que podemos controlar indirectamente.